L'objectif est de prédire le vainqueur d'un match de tennis à partir des données de matchs de tennis ayant eu lieu entre 1950 et 2024.

## Imports

In [3]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,balanced_accuracy_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import requests
from bs4 import BeautifulSoup
import random
np.set_printoptions(threshold=np.inf)

In [280]:
directory = '/Users/stanislas/Downloads/To-Import-Model'

dfs = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        filepath = os.path.join(directory, filename)
        data = pd.read_csv(filepath)
        dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

In [281]:
#Players information
players = pd.read_csv('atp_players.csv')

In [282]:
#Top500 references
top500=pd.read_excel('Top500-Modified.xlsx')

## DF reprocessing

### Basic information

In [283]:
df.dtypes

tourney_id             object
tourney_name           object
surface                object
draw_size             float64
tourney_level          object
tourney_date            int64
match_num               int64
winner_id               int64
winner_seed           float64
winner_entry           object
winner_name            object
winner_hand            object
winner_ht             float64
winner_ioc             object
winner_age            float64
loser_id                int64
loser_seed            float64
loser_entry            object
loser_name             object
loser_hand             object
loser_ht              float64
loser_ioc              object
loser_age             float64
score                  object
best_of                 int64
round                  object
minutes               float64
w_ace                 float64
w_df                  float64
w_svpt                float64
w_1stIn               float64
w_1stWon              float64
w_2ndWon              float64
w_SvGms   

In [284]:
df.describe()

draw_size  tourney_date      match_num      winner_id  \
count  191085.000000  1.919200e+05  191920.000000  191920.000000   
mean       53.088479  1.993711e+07      79.398656  104499.014537   
std        36.645414  1.581156e+05     111.963129   13664.799449   
min         2.000000  1.967123e+07       1.000000  100001.000000   
25%        32.000000  1.980051e+07      11.000000  100417.000000   
50%        32.000000  1.993052e+07      25.000000  101733.000000   
75%        64.000000  2.007032e+07      94.000000  103990.000000   
max       128.000000  2.023113e+07    1701.000000  212428.000000   

        winner_seed      winner_ht     winner_age       loser_id  \
count  70926.000000  175012.000000  190609.000000  191920.000000   
mean       6.308279     184.491618      25.670153  105686.966012   
std        5.547887       6.672384       4.053105   16655.971639   
min        1.000000     160.000000      14.300000  100001.000000   
25%        2.000000     180.000000      22.700000  100523.000000   
50%        5.000000     185.000000      25.300000  101900.000000   
75%        8.000000     188.000000      28.200000  104338.000000   
max       35.000000     211.000000      58.700000  212437.000000   

         loser_seed       loser_ht  ...       l_1stIn      l_1stWon  \
count  36099.000000  162257.000000  ...  95034.000000  95034.000000   
mean       7.711017     184.261696  ...     48.089421     31.962361   
std        6.055301       6.658745  ...     19.378619     14.447415   
min        1.000000     160.000000  ...      0.000000      0.000000   
25%        4.000000     180.000000  ...     34.000000     22.000000   
50%        6.000000     185.000000  ...     45.000000     30.000000   
75%       10.000000     188.000000  ...     58.000000     40.000000   
max       35.000000     211.000000  ...    328.000000    284.000000   

           l_2ndWon       l_SvGms     l_bpSaved     l_bpFaced    winner_rank  \
count  95034.000000  95035.000000  95034.000000  95034.000000  156159.000000   
mean      14.972936     12.206229      4.808942      8.730570      75.246960   
std        7.207299      4.131171      3.275134      4.132557     120.956559   
min        0.000000      0.000000     -6.000000      0.000000       1.000000   
25%       10.000000      9.000000      2.000000      6.000000      17.000000   
50%       14.000000     11.000000      4.000000      8.000000      45.000000   
75%       19.000000     15.000000      7.000000     11.000000      86.000000   
max      101.000000     91.000000     28.000000     38.000000    2101.000000   

       winner_rank_points     loser_rank  loser_rank_points  
count       108936.000000  147788.000000      107308.000000  
mean          1375.909663     112.905710         865.976908  
std           1729.720178     162.603579         993.837316  
min              0.000000       1.000000           0.000000  
25%            494.000000      37.000000         365.000000  
50%            850.000000      70.000000         635.000000  
75%           1540.000000     118.000000        1015.000000  
max          16950.000000    2159.000000       16950.000000  

[8 rows x 35 columns]

In [285]:
df.isna().sum()

tourney_id                 0
tourney_name               0
surface                 2990
draw_size                835
tourney_level              0
tourney_date               0
match_num                  0
winner_id                  0
winner_seed           120994
winner_entry          175177
winner_name                0
winner_hand               12
winner_ht              16908
winner_ioc                 8
winner_age              1311
loser_id                   0
loser_seed            155821
loser_entry           163496
loser_name                 0
loser_hand                49
loser_ht               29663
loser_ioc                 72
loser_age               4657
score                      9
best_of                    0
round                      0
minutes                99653
w_ace                  96885
w_df                   96886
w_svpt                 96886
w_1stIn                96886
w_1stWon               96886
w_2ndWon               96886
w_SvGms                96885
w_bpSaved     

### Keep only significant columns

In [286]:
df = df[['tourney_id','tourney_name','surface','draw_size','tourney_level','tourney_date',
         'match_num','winner_id','winner_hand','winner_ht','winner_age','loser_id','loser_hand',
        'loser_ht','loser_age','best_of','round','winner_rank','loser_rank','winner_rank_points','loser_rank_points']]

## Data Format

### Datetime format for tourney_date

In [287]:
df['tourney_date']=df['tourney_date'].fillna(0)

df['date_str'] = df['tourney_date'].astype(int).astype(str)

def convert_to_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y%m%d').strftime('%Y-%m-%d')
    except ValueError:
        return None  
        
# tourney_date should get date format
df['tourney_date'] = df['date_str'].apply(convert_to_date)

# We do not need the date string column anymore
df.drop(columns=['date_str'], inplace=True)

### Reducing values in the column tourney_name

In [288]:
sorted(df['tourney_name'].unique())

['ATP Rio de Janeiro',
 'Aberavon',
 'Acapulco',
 'Adelaide',
 'Adelaide 1',
 'Adelaide 2',
 'Adelaide-2',
 'Aix en Provence',
 'Aix-en-Provence',
 'Alamo WCT',
 'Albany',
 'Algiers',
 'Amersfoort',
 'Amsterdam',
 'Amsterdam WCT',
 'Anaheim',
 'Ancona',
 'Antalya',
 'Antwerp',
 'Aptos',
 'Astana',
 'Athens',
 'Athens Olympics',
 'Atlanta',
 'Atlanta Olympics',
 'Atlanta WCT',
 'Atp Cup',
 'Auckland',
 'Australian Chps.',
 'Australian Open',
 'Australian Open-2',
 'Australian Round Robin',
 'Aviles',
 'Bahia',
 'Bakersfield WCT',
 'Baltimore',
 'Baltimore WCT',
 'Bangalore',
 'Bangkok',
 'Banja Luka',
 'Barcelona',
 'Barcelona 2',
 'Barcelona Nationals',
 'Barcelona Olympics',
 'Barcelona WCT',
 'Bari',
 'Barranquilla',
 'Basel',
 'Bastad',
 'Bastad 1',
 'Bastad WCT',
 'Beckenham',
 'Beijing',
 'Beijing Olympics',
 'Belgrade',
 'Belgrade ',
 'Belgrade 2',
 'Berkeley',
 'Berlin',
 'Bermuda',
 'Binghamton',
 'Binghamton NTL',
 'Birmingham',
 'Birmingham WCT',
 'Bloemfontein',
 'Boca Raton

In [289]:
# Some tournaments appear with different names. We should make it easier
def replace_tourney_name(df):
    mask1 = df['tourney_name'].str.contains('Davis Cup')
    df.loc[mask1, 'tourney_name'] = 'Davis Cup'
    
    mask2 = df['tourney_name'].str.contains('Olympics')
    df.loc[mask2, 'tourney_name'] = 'Olympics'
    
    mask3 = df['tourney_name'].str.contains('Australian Open')
    df.loc[mask3, 'tourney_name'] = 'Australian Open'
    
    mask4 = df['tourney_name'].str.contains('Adelaide')
    df.loc[mask4, 'tourney_name'] = 'Adelaide'
    
    mask5 = df['tourney_name'].str.contains('Washington')
    df.loc[mask5, 'tourney_name'] = 'Washington'
    
    mask6 = df['tourney_name'].str.contains('Barcelona')
    df.loc[mask6, 'tourney_name'] = 'Barcelona'
    
    mask7 = df['tourney_name'].str.contains('Belgrade')
    df.loc[mask7, 'tourney_name'] = 'Belgrade'
    
    mask8 = df['tourney_name'].str.contains('Cincinnati')
    df.loc[mask8, 'tourney_name'] = 'Cincinnati'
    
    mask9 = df['tourney_name'].str.contains('Hamburg')
    df.loc[mask9, 'tourney_name'] = 'Hamburg'
    
    mask10 = df['tourney_name'].str.contains('Indian Wells')
    df.loc[mask10, 'tourney_name'] = 'Indian Wells'
    
    mask11 = df['tourney_name'].str.contains('Madrid')
    df.loc[mask11, 'tourney_name'] = 'Madrid'
    
    mask12 = df['tourney_name'].str.contains('Miami')
    df.loc[mask12, 'tourney_name'] = 'Miami'
    
    mask13 = df['tourney_name'].str.contains('Monte Carlo')
    df.loc[mask13, 'tourney_name'] = 'Monte Carlo'
    
    mask14 = df['tourney_name'].str.contains('New York')
    df.loc[mask14, 'tourney_name'] = 'New York'
    
    mask15 = df['tourney_name'].str.contains('Rome')
    df.loc[mask15, 'tourney_name'] = 'Rome'
    
    mask16 = df['tourney_name'].str.contains('Shanghai')
    df.loc[mask16, 'tourney_name'] = 'Shanghai'
    
    mask17 = df['tourney_name'].str.contains('Toronto')
    df.loc[mask17, 'tourney_name'] = 'Toronto'
    
    mask18 = df['tourney_name'].str.contains('Janeiro')
    df.loc[mask18, 'tourney_name'] = 'Rio de Janeiro'
    
    return df

In [290]:
df = replace_tourney_name(df)

### Handle missing values

#### For surface

In [291]:
df[df['surface'].isna()]['tourney_name'].unique()

array(['Davis Cup', 'Perth', 'Bristol', 'Manchester', 'Cape Town',
       'Mexico City', 'Calcutta', 'Port Elizabeth', 'San Juan', 'Phoenix',
       'Cannes', 'Los Angeles SoCal Chps', 'Oakland', 'Woodside',
       'Helsinki', 'Saltsjoebaden', 'San Antonio Collegiate',
       'Montana Vermala', 'Senigallia', 'Quebec City', 'Bucharest',
       'Mamaia', 'Las Vegas', 'Madrid', 'Valencia', 'Cannes Chps',
       'Hanau', 'Tournament of Champions WCT', 'Roanoke', 'Dublin',
       'Cedar Grove', 'Christchurch', 'Oslo', 'New Orleans WCT', 'Omaha',
       'Paramus', 'Calgary', 'Salt Lake City', 'Washington', 'Tokyo WCT',
       'Tokyo', 'Freeport', 'Shreveport', 'New York', 'Istanbul',
       'Quebec WCT', 'Vancouver WCT', 'Casablanca WCT', 'Seattle',
       'Tanglewood', 'Montreal WCT', 'Alamo WCT', 'Los Angeles',
       'Des Moines', 'Kansas City', 'Sacramento', 'Quebec', 'New Delhi',
       'Djkarta'], dtype=object)

In [292]:
# Get NaN values for surface
missing_surface = df[df['surface'].isnull()]

# Browse lines of this df
for index, row in missing_surface.iterrows():
    # Get tourney name value
    tourney_name = row['tourney_name']
    
    # Get similar tourney names containing a value where surface is NaN
    matching_rows = df[(df['tourney_name'] == tourney_name) & ~(df['surface'].isnull())]
    
    # Take the first corresponding value and put it in the missing surface cell
    if not matching_rows.empty:
        surface_value = matching_rows.iloc[0]['surface']
        df.at[index, 'surface'] = surface_value

#### For winner height

In [293]:
# Get NaN values for winner_ht
missing_winner_ht = df[df['winner_ht'].isnull()]

# Browse lines of this df
for index, row in missing_winner_ht.iterrows():
    # Get winner id value
    winner_id = row['winner_id']
    
    # Get the value in the players dataframe for corresponding id
    player_info = players[players['player_id'] == winner_id]
    
    if not player_info.empty and not pd.isnull(player_info.iloc[0]['height']):
        height_value = player_info.iloc[0]['height']
        df.at[index, 'winner_ht'] = height_value

#### For loser height

In [294]:
# Get NaN values for loser_ht
missing_loser_ht = df[df['loser_ht'].isnull()]

# Browse lines of this df
for index, row in missing_loser_ht.iterrows():
    # # Get loser height value
    loser_id = row['loser_id']
    
    # Get the value in the players dataframe for corresponding id
    player_info = players[players['player_id'] == loser_id]
    
    if not player_info.empty and not pd.isnull(player_info.iloc[0]['height']):
        height_value = player_info.iloc[0]['height']
        df.at[index, 'loser_ht'] = height_value

#### For loser age

In [295]:
# Get NaN values for loser_age
players['dob'] = pd.to_datetime(players['dob'])

df['tourney_date'] = pd.to_datetime(df['tourney_date'])

# Calculate age by substracting tourney date and date of birth
def calculate_age(row):
    if pd.isnull(row['loser_age']):
        loser_id = row['loser_id']
        player_info = players[players['player_id'] == loser_id]
        if not player_info.empty and not pd.isnull(player_info.iloc[0]['dob']):
            dob = player_info.iloc[0]['dob']
            tourney_date = row['tourney_date']
            return (tourney_date - dob).days // 365
    return row['loser_age']

df['loser_age'] = df.apply(calculate_age, axis=1)

### Calculate means of winner heights values for every combination of loser height, tourney name and round

In [296]:
# Average winner height for different loser height, tourney and round
winner_ht_means = pd.DataFrame(df.groupby(['loser_ht','tourney_name','round'])['winner_ht'].mean())
winner_ht_means

winner_ht
loser_ht tourney_name round           
160.0    Athens       R16        180.0
                      R32        180.0
         Barcelona    R32        180.0
                      R64        188.0
         Bari         R16        185.0
...                                ...
211.0    Washington   R32        193.0
                      R64        191.5
         Wimbledon    R128       180.0
                      R32        196.0
                      R64        188.0

[18034 rows x 1 columns]

We can replace NaN winner_ht with similar values for this columnes with the means

In [297]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same loser height, tourney and round
merged_df = df.merge(winner_ht_means, how='left', on=['loser_ht', 'tourney_name', 'round'], suffixes=('', '_mean'))

df['winner_ht'] = df['winner_ht'].fillna(merged_df['winner_ht_mean'])

In [298]:
# Average loser height for different winner height, tourney and round
loser_ht_means = pd.DataFrame(df.groupby(['winner_ht','tourney_name','round'])['loser_ht'].mean())
loser_ht_means

loser_ht
winner_ht tourney_name round          
160.0     Athens       R32       185.0
          Barcelona    R64       183.0
          Bari         R32       183.0
          Davis Cup    RR          NaN
          Monte Carlo  R64         NaN
...                                ...
211.0     Us Open      R64       185.0
          Washington   R32       185.5
                       R64       201.0
          Wimbledon    R128      183.0
                       R64       183.0

[20323 rows x 1 columns]

In [299]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same winner height, tourney and round
merged_df = df.merge(loser_ht_means, how='left', on=['winner_ht', 'tourney_name', 'round'], suffixes=('', '_mean'))

df['loser_ht'] = df['loser_ht'].fillna(merged_df['loser_ht_mean'])

In [300]:
# Average loser rank for different tourney and round
loser_rank_means = pd.DataFrame(df.groupby(['tourney_name','round'])['loser_rank'].mean())
loser_rank_means

loser_rank
tourney_name    round            
Aberavon        F             NaN
                QF            NaN
                R16           NaN
                R32           NaN
                SF            NaN
...                           ...
s Hertogenbosch F      107.826087
                QF      74.854167
                R16     98.296296
                R32    128.919540
                SF      62.319149

[2611 rows x 1 columns]

In [301]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same tourney and round
merged_df = df.merge(loser_rank_means, how='left', on=['tourney_name','round'], suffixes=('', '_mean'))

df['loser_rank'] = df['loser_rank'].fillna(merged_df['loser_rank_mean'])

In [302]:
# Average winner rank for different tourney and round
winner_rank_means = pd.DataFrame(df.groupby(['tourney_name','round'])['winner_rank'].mean())
winner_rank_means

winner_rank
tourney_name    round             
Aberavon        F              NaN
                QF             NaN
                R16            NaN
                R32            NaN
                SF             NaN
...                            ...
s Hertogenbosch F        49.500000
                QF       70.180851
                R16      72.542105
                R32      91.302594
                SF       78.042553

[2611 rows x 1 columns]

In [303]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same tourney and round
merged_df = df.merge(winner_rank_means, how='left', on=['tourney_name','round'], suffixes=('', '_mean'))

df['winner_rank'] = df['winner_rank'].fillna(merged_df['winner_rank_mean'])

In [304]:
# Average loser rank points for different tourney and round
loser_rank__points_means = pd.DataFrame(df.groupby(['tourney_name','round'])['loser_rank_points'].mean())
loser_rank__points_means

loser_rank_points
tourney_name    round                   
Aberavon        F                    NaN
                QF                   NaN
                R16                  NaN
                R32                  NaN
                SF                   NaN
...                                  ...
s Hertogenbosch F            1330.217391
                QF            948.729167
                R16           791.063492
                R32           619.583333
                SF           1115.000000

[2611 rows x 1 columns]

In [305]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same tourney and round
merged_df = df.merge(loser_rank__points_means, how='left', on=['tourney_name','round'], suffixes=('', '_mean'))

df['loser_rank_points'] = df['loser_rank_points'].fillna(merged_df['loser_rank_points_mean'])

In [306]:
# Average winner rank points for different tourney and round
winner_rank__points_means = pd.DataFrame(df.groupby(['tourney_name','round'])['winner_rank_points'].mean())
winner_rank__points_means

winner_rank_points
tourney_name    round                    
Aberavon        F                     NaN
                QF                    NaN
                R16                   NaN
                R32                   NaN
                SF                    NaN
...                                   ...
s Hertogenbosch F             1524.291667
                QF            1272.159574
                R16           1108.742105
                R32            823.449568
                SF            1429.319149

[2611 rows x 1 columns]

In [307]:
# Merge the new df with the current one and fill the NaN with rows corresponding to the same tourney and round
merged_df = df.merge(winner_rank__points_means, how='left', on=['tourney_name','round'], suffixes=('', '_mean'))

df['winner_rank_points'] = df['winner_rank_points'].fillna(merged_df['winner_rank_points_mean'])

### Fill remaining empty rank points

In [308]:
df.isna().sum()

tourney_id                0
tourney_name              0
surface                 987
draw_size               835
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_hand              12
winner_ht             11394
winner_age             1311
loser_id                  0
loser_hand               49
loser_ht              12787
loser_age              4369
best_of                   0
round                     0
winner_rank            6157
loser_rank             6294
winner_rank_points    23506
loser_rank_points     23506
dtype: int64

#### Calculate the average difference between winner rank points and winner rank

In [309]:
np.mean(df['winner_rank_points']/df['winner_rank'].mean())

16.762872659327172

#### Calculate the average difference between loser rank points and loser rank

In [310]:
np.mean(df['loser_rank_points']/df['loser_rank'].mean())

6.829995326384309

In [311]:
# Get empty lines from winner and loser rank points
missing_points_rows = df[df['winner_rank_points'].isnull() & df['loser_rank_points'].isnull()]

# Multiply the corresponding rank by the mean multiple for winner and loser rank points
missing_points_rows['loser_rank_points'] = missing_points_rows['loser_rank'] * 6.83
missing_points_rows['winner_rank_points'] = missing_points_rows['winner_rank'] * 16.76

# winner_rank_points should always be superior to loser_rank_points
missing_points_rows.loc[missing_points_rows['winner_rank_points'] <= missing_points_rows['loser_rank_points'], 'winner_rank_points'] = missing_points_rows['loser_rank_points'] + 1

# Update df
df.update(missing_points_rows)

/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_11940/2531697370.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_points_rows['loser_rank_points'] = missing_points_rows['loser_rank'] * 6.83
/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_11940/2531697370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_points_rows['winner_rank_points'] = missing_points_rows['winner_rank'] * 16.76


#### Drop remaining NaN

In [312]:
df=df.dropna(subset=['surface','winner_ht','winner_age','loser_ht','loser_age','winner_rank','loser_hand','loser_rank','winner_hand','winner_rank_points','loser_rank_points'])

In [313]:
df.isna().sum()

tourney_id             0
tourney_name           0
surface                0
draw_size             68
tourney_level          0
tourney_date           0
match_num              0
winner_id              0
winner_hand            0
winner_ht              0
winner_age             0
loser_id               0
loser_hand             0
loser_ht               0
loser_age              0
best_of                0
round                  0
winner_rank            0
loser_rank             0
winner_rank_points     0
loser_rank_points      0
dtype: int64

## Second column filter to keep only the essentials

In [314]:
df=df[['tourney_name','surface','round','winner_id','winner_hand','winner_ht','winner_age','winner_rank','winner_rank_points',
    'loser_id','loser_hand','loser_ht','loser_age','loser_rank','loser_rank_points']]

## Creation of 2 dataframes : one for victories and one for losses

By doing this, I duplicate my number of columns and put myself in a player versus opponent perspective. Each match will be interpreted from the winner's point of view as a victory and then from the loser's point of view as a defeat.

In [315]:
df_winner = df.copy()
df_loser = df.copy()

In [316]:
df.columns

Index(['tourney_name', 'surface', 'round', 'winner_id', 'winner_hand',
       'winner_ht', 'winner_age', 'winner_rank', 'winner_rank_points',
       'loser_id', 'loser_hand', 'loser_ht', 'loser_age', 'loser_rank',
       'loser_rank_points'],
      dtype='object')

### Rename columns to have one player and his opponent

In [317]:
df_winner=df_winner.rename(columns={'winner_id':'Player','winner_hand':'Player_Hand','winner_ht':'Player_Height',
                         'winner_age':'Player_Age','winner_rank':'Player_Rank','winner_rank_points':'Player_Rank_Points',
                        'loser_id':'Opponent','loser_hand':'Opponent_Hand',
                          'loser_ht':'Opponent_Height','loser_age':'Opponent_Age','loser_rank':'Opponent_Rank',
                            'loser_rank_points':'Opponent_Rank_Points'})

#### Players here are the winners so Result=1

In [318]:
df_winner['Result']=1

### Rename columns to have one player and his opponent

In [319]:
df_loser=df_loser.rename(columns={'loser_id':'Player','loser_hand':'Player_Hand','loser_ht':'Player_Height',
                         'loser_age':'Player_Age','loser_rank':'Player_Rank','loser_rank_points':'Player_Rank_Points'
                                  ,'winner_id':'Opponent','winner_hand':'Opponent_Hand',
                          'winner_ht':'Opponent_Height','winner_age':'Opponent_Age','winner_rank':'Opponent_Rank','winner_rank_points':'Opponent_Rank_Points'})

#### Players here are the losers so Result=0

In [320]:
df_loser['Result']=0

In [321]:
df_winner

tourney_name surface round  Player Player_Hand  Player_Height  \
0          Brisbane    Hard     F  105453           R          178.0   
1          Brisbane    Hard    SF  106421           R          198.0   
2          Brisbane    Hard    SF  105453           R          178.0   
3          Brisbane    Hard    QF  104542           R          188.0   
4          Brisbane    Hard    QF  106421           R          198.0   
...             ...     ...   ...     ...         ...            ...   
191915  Tour Finals    Hard    SF  104925           R          188.0   
191916  Tour Finals    Hard     F  104925           R          188.0   
191917    Davis Cup    Clay    RR  104527           R          183.0   
191918    Davis Cup    Clay    RR  104792           R          193.0   
191919    Davis Cup    Clay    RR  103819           R          185.0   

        Player_Age  Player_Rank  Player_Rank_Points  Opponent Opponent_Hand  \
0             29.0          9.0              3590.0    106421             R   
1             22.8         16.0              1977.0    104542             R   
2             29.0          9.0              3590.0    104871             R   
3             33.7        239.0               200.0    200282             R   
4             22.8         16.0              1977.0    105683             R   
...            ...          ...                 ...       ...           ...   
191915        27.4          1.0             10010.0    105453             R   
191916        27.4          1.0             10010.0    103819             R   
191917        29.6          4.0              5295.0    104542             R   
191918        28.2         19.0              1825.0    103819             R   
191919        33.2          2.0              9700.0    104755             R   

        Opponent_Height  Opponent_Age  Opponent_Rank  Opponent_Rank_Points  \
0                 198.0          22.8           16.0                1977.0   
1                 188.0          33.7          239.0                 200.0   
2                 188.0          31.8           40.0                1050.0   
3                 183.0          19.8           31.0                1298.0   
4                 196.0          28.0           18.0                1855.0   
...                 ...           ...            ...                   ...   
191915            178.0          24.8            5.0                4625.0   
191916            185.0          33.2            2.0                8700.0   
191917            188.0          29.5           12.0                2740.0   
191918            185.0          33.2            2.0                9700.0   
191919            185.0          28.4           26.0                1350.0   

        Result  
0            1  
1            1  
2            1  
3            1  
4            1  
...        ...  
191915       1  
191916       1  
191917       1  
191918       1  
191919       1  

[173370 rows x 16 columns]

In [322]:
df_loser=df_loser[['tourney_name','surface','round','Player','Player_Hand','Player_Height','Player_Age','Player_Rank','Player_Rank_Points','Opponent','Opponent_Hand','Opponent_Height','Opponent_Age','Opponent_Rank','Opponent_Rank_Points','Result']]

In [323]:
df_loser

tourney_name surface round  Player Player_Hand  Player_Height  \
0          Brisbane    Hard     F  106421           R          198.0   
1          Brisbane    Hard    SF  104542           R          188.0   
2          Brisbane    Hard    SF  104871           R          188.0   
3          Brisbane    Hard    QF  200282           R          183.0   
4          Brisbane    Hard    QF  105683           R          196.0   
...             ...     ...   ...     ...         ...            ...   
191915  Tour Finals    Hard    SF  105453           R          178.0   
191916  Tour Finals    Hard     F  103819           R          185.0   
191917    Davis Cup    Clay    RR  104542           R          188.0   
191918    Davis Cup    Clay    RR  103819           R          185.0   
191919    Davis Cup    Clay    RR  104755           R          185.0   

        Player_Age  Player_Rank  Player_Rank_Points  Opponent Opponent_Hand  \
0             22.8         16.0              1977.0    105453             R   
1             33.7        239.0               200.0    106421             R   
2             31.8         40.0              1050.0    105453             R   
3             19.8         31.0              1298.0    104542             R   
4             28.0         18.0              1855.0    106421             R   
...            ...          ...                 ...       ...           ...   
191915        24.8          5.0              4625.0    104925             R   
191916        33.2          2.0              8700.0    104925             R   
191917        29.5         12.0              2740.0    104527             R   
191918        33.2          2.0              9700.0    104792             R   
191919        28.4         26.0              1350.0    103819             R   

        Opponent_Height  Opponent_Age  Opponent_Rank  Opponent_Rank_Points  \
0                 178.0          29.0            9.0                3590.0   
1                 198.0          22.8           16.0                1977.0   
2                 178.0          29.0            9.0                3590.0   
3                 188.0          33.7          239.0                 200.0   
4                 198.0          22.8           16.0                1977.0   
...                 ...           ...            ...                   ...   
191915            188.0          27.4            1.0               10010.0   
191916            188.0          27.4            1.0               10010.0   
191917            183.0          29.6            4.0                5295.0   
191918            193.0          28.2           19.0                1825.0   
191919            185.0          33.2            2.0                9700.0   

        Result  
0            0  
1            0  
2            0  
3            0  
4            0  
...        ...  
191915       0  
191916       0  
191917       0  
191918       0  
191919       0  

[173370 rows x 16 columns]

## Concat to get the final dataframe

In [324]:
df=pd.concat([df_winner,df_loser])

### Instead of leaving two columns for the same feature, create a differential.

In [325]:
df['Hand_Opposition'] = df['Player_Hand'] + ' vs ' + df['Opponent_Hand']

In [326]:
df['Height_Differential'] = df['Player_Height']-df['Opponent_Height']

In [327]:
df['Age_Difference'] = df['Player_Age']-df['Opponent_Age']

In [328]:
df['Rank_Differential'] = df['Player_Rank']-df['Opponent_Rank']

In [329]:
df['Ranking_Points_Differential'] = df['Player_Rank_Points']-df['Opponent_Rank_Points']

In [330]:
df=df[['tourney_name','surface','Player','Opponent','Hand_Opposition','Height_Differential','Age_Difference','Rank_Differential','Ranking_Points_Differential','Result']]

In [331]:
df.to_csv('df.csv')

## Players reprocessing

### Sélection des joueurs présents dans top500

#### Add Full Name

In [32]:
players['Name'] = players['name_first'] + ' ' + players['name_last']

#### Keep only the players in the ATP Top 500 beginning of 2024

In [167]:
players = players[players['Name'].isin(top500['Player'])]

In [171]:
players=players.rename(columns={'Name':'Player'})

### Get full data : Players characteristics and Rankings from ATP

In [172]:
full_data=pd.merge(players,top500,on='Player',how='left')

In [174]:
full_data.to_csv('players_information.csv')

### Encodage des variables textuelles devant avoir les mêmes valeurs entre colonnes

In [ ]:
# Créer un ensemble de tous les ID uniques
unique_ids = set(df['Player_1_id']).union(set(df['Player_2_id'])).union(set(df['Winner']))

# Créer un dictionnaire pour mapper chaque ID unique à un nombre unique
id_to_number = {id_: i for i, id_ in enumerate(unique_ids)}

# Remplacer les valeurs des colonnes Player_1_id, Player_2_id et Winner par les nombres correspondants
df['Player_1_id'] = df['Player_1_id'].map(id_to_number)
df['Player_2_id'] = df['Player_2_id'].map(id_to_number)
df['Winner'] = df['Winner'].map(id_to_number)

In [435]:
numeric = ['Height_Differential','Age_Difference', 'Ranking_Gap', 'Ranking_Points_Difference']
string = ['tourney_name','surface','Hand_Opposition']

In [436]:
for col in df.columns:
    if col in numeric:
        # Standardisation des données numériques
        scaler = StandardScaler()
        df[col] = scaler.fit_transform(df[[col]])
    elif col in string:
        # Encodage des données de chaîne
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])

### Remise en forme de Winner

In [452]:
df['Winner'] = df['Winner'].replace({29: 0, 378: 1})

## Récupération de df

In [453]:
df

tourney_name  surface  Player_1_id  Player_2_id  Hand_Opposition  \
0                 51        3         1724         1926                8   
8               1857        2          823          818                8   
11              1789        3          205          537                8   
13                55        3          907          130                4   
14              1790        1          159           17                8   
...              ...      ...          ...          ...              ...   
210737          1857        1          221          986                8   
210738          1886        3         1995          259                8   
210739          1876        3         2023         1812                8   
210740            17        3          626          771                8   
210741          1789        3          893          912                8   

        Height_Differential  Age_Difference  Ranking_Gap  \
0                 -2.284599        1.221184    -0.048692   
8                  1.424725       -0.023975    -0.209432   
11                -0.539035        0.948806     0.031678   
13                -0.539035       -1.911170    -0.136369   
14                 0.006454       -0.471454     0.228950   
...                     ...             ...          ...   
210737             1.097432        1.765942    -0.172901   
210738             1.642921       -0.004520    -0.245964   
210739            -0.539035       -0.413088    -0.516300   
210740             0.551943        0.267859    -1.363840   
210741             0.551943        0.034392     0.404303   

        Ranking_Points_Difference  Winner  
0                       -0.048692       0  
8                       -0.209432       0  
11                       0.031678       0  
13                      -0.136369       1  
14                       0.228950       0  
...                           ...     ...  
210737                  -0.172901       0  
210738                  -0.245964       0  
210739                  -0.516300       1  
210740                  -1.363840       0  
210741                   0.404303       0  

[112951 rows x 10 columns]

In [454]:
data = df.copy()

In [455]:
data.to_csv('data.csv')

In [456]:
df = data

In [4]:
os.system('jupyter nbconvert --to html Pre-processing.ipynb')

[NbConvertApp] Converting notebook Pre-processing.ipynb to html
[NbConvertApp] Writing 460288 bytes to Pre-processing.html


0